<a href="https://colab.research.google.com/github/safal207/hello-world/blob/master/%D0%A1%D0%B0%D1%84%D0%BE%D0%BD%D0%BE%D0%B2_%D0%90_%D0%90_%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Рисовать графики сразу же
%matplotlib inline

import re # регулярные выражения

# Статический вывод графики (графики отображаются в той ячейке, в которой используется plt.show())
import pandas as pd # Импортируем библиотеку обработки и анализа данных pandas
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
import datetime # Импортируем модуль datetime для обработки данных в формете даты
import numpy as np # Импортируем библиотеку numpy
from google.colab import files # Импортируем библиотеку files для работы с файлами

#Задание. База резюме headhunter. 

##Загружаем базу



In [ ]:
# Для загрузки данных используем drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
fixed_df = pd.read_csv('/content/drive/My Drive/Базы/hh (1).csv',sep=',') # Путь, куда вы скачали файл

##Содержимое базы

In [ ]:
fixed_df.head()

,Unnamed: 0,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,0,"Мужчина , 42 года , родился 6 октября 1976",27 000 руб.,Старший администратор,"Липецк , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 6 лет 1 месяц\n\nСтарший администр...,Трц Ривьера,Старший администратор по сохраности активов,Среднее специальное образование 1994 Спту 26,28.04.2019 12:17,Имеется собственный автомобиль
1,1,"Мужчина , 41 год , родился 14 января 1978",60 000 руб.,Старший IT инженер,"Сочи , не готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день",Опыт работы 19 лет 2 месяца\n\nСтарший IT инже...,"Эскорт-Центр, Москва, ЗАО",Старший инженер сервисного центра,Высшее образование 1999 СОЧИНСКИЙ ГОС. УНИВЕР...,28.04.2019 11:44,Не указано
2,2,"Мужчина , 44 года , родился 21 февраля 1975",65 000 руб.,"Системный администратор, Системный инженер","Липецк , готов к переезду , готов к командировкам",полная занятость,"удаленная работа, полный день, вахтовый метод",Опыт работы 19 лет 9 месяцев\n\nСистемный адми...,"СПССПК ""ЭКОПТИЦА""",Системный администратор,Высшее образование 2003 Московский государств...,28.04.2019 10:36,Не указано
3,3,"Мужчина , 43 года , родился 9 февраля 1976",70 000 руб.,"Web-программист, Web-администратор, Web-мастер","Москва , м. Марьино , не готов к переезду , г...",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы 5 лет 9 месяцев\n\nWeb-программист...,"ГБУ ""МДОО""",специалист отдела интернет-проектов,Высшее образование 2002 Российский государств...,28.04.2019 10:29,Не указано
4,4,"Мужчина , 39 лет , родился 14 июня 1979",45 000 руб.,Старший системный администратор \ начальник от...,"Тула , не готов к переезду , готов к редким ко...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 5 месяцев\n\nСтарший систем...,"ООО ""СервисСофт""",Старший системный администратор,Высшее образование 2007 Российский государств...,28.04.2019 10:15,Не указано


In [ ]:
# Количество резюме и критерии
print(fixed_df.shape)

(66945, 13)


In [ ]:
# Пример данных
n = 1 # Берем первое резюме
for i in range(len(fixed_df.values[n])): # Выводим значение каждой графы резюме
  print(i, " ", fixed_df.values[n][i])

0   1
1   Мужчина ,  41 год , родился 14 января 1978
2   60 000 руб.
3   Старший IT инженер
4   Сочи , не готов к переезду , готов к командировкам
5    частичная занятость, полная занятость
6    гибкий график, полный день
7   Опыт работы 19 лет 2 месяца

Старший IT инженер 60 000 руб. Информационные технологии, интернет, телеком Сетевые технологии Системная интеграция Системный администратор Занятость: частичная занятость, полная занятость График работы: гибкий график, полный день Опыт работы 19 лет 2 месяца Июнь 2012 — по настоящее время 6 лет 11 месяцев Эскорт-Центр, Москва, ЗАО Краснодарский край , www.escort-center.ru Информационные технологии, системная интеграция, интернет ... Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг Услуги для бизнеса Безопасность, охранная деятельность Старший инженер сервисного центра Обязанности: Обеспечение бесперебойного функционирования информационных систем управления. Настройка и администрирование 

In [ ]:
##############################################
####Убрать кодировку в описаниях \xa0 \n
##############################################
fixed_df = fixed_df.drop(fixed_df.columns[0],axis = 1) # Убираем первый столбец с продублированными индексами. Он нам не нужен

i=0
for j in range(12):  # Для каждого из столбцов таблицы
    for i in range(fixed_df.shape[0]): # По всем резюме
        if type(fixed_df.values[i][j])!=float: # если данные текстовые, то заменяем фрагменты кодировки ASCII на отсутствие символов
            fixed_df.values[i][j] = fixed_df.values[i][j].replace("\xa0","")
            fixed_df.values[i][j] = fixed_df.values[i][j].replace("\n"," ")


In [ ]:
# Пример данных
n = 1 # Берем первое резюме
for i in range(len(fixed_df.values[n])): # Выводим значение каждой графы резюме
  print(i, " ", fixed_df.values[n][i])

0   Мужчина ,  41год , родился 14 января 1978
1   60000 руб.
2   Старший IT инженер
3   Сочи , не готов к переезду , готов к командировкам
4    частичная занятость, полная занятость
5    гибкий график, полный день
6   Опыт работы 19 лет 2 месяца  Старший IT инженер 60000 руб. Информационные технологии, интернет, телеком Сетевые технологии Системная интеграция Системный администратор Занятость: частичная занятость, полная занятость График работы: гибкий график, полный день Опыт работы 19 лет 2 месяца Июнь2012 — понастоящеевремя 6 лет 11 месяцев Эскорт-Центр, Москва, ЗАО Краснодарский край , www.escort-center.ru Информационные технологии, системная интеграция, интернет ... Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг Услуги для бизнеса Безопасность, охранная деятельность Старший инженер сервисного центра Обязанности: Обеспечение бесперебойного функционирования информационных систем управления. Настройка и администрирование Windows XP, 

##Парсим базу


Версия для хранения

In [ ]:
# Данные о поле и возрасте
def getParameterSexAge(arg):
  out = 0
  # Если М, то 1. По умолчанию 0 - Ж
  if "Муж" in arg:
    sex='М'
  elif "Жен" in arg:
    sex = 'Ж'
  else:
    sex = "Не указан"
  # Текущий год - год рождения
  year_tec = 2020
  if (len(arg) > 7):
    out = year_tec - int(re.findall(r'\d{4}', arg)[0])

  return sex, str(out)

# Зарплата
def getParameterSalary(arg):
    global salaryStr, num # т.к. используем, меняем в функции
    # Сначала получаем чистое число, убираем лишние знаки
    salaryStr = 0
    arg=str(arg)  
    salaryStr = int(re.sub(r'[^0-9]+', '', arg))
    curr_ = re.sub('[0-9]','',arg).replace('.','')

    # Конвертируем в рубли, если валюта
    if 'USD' in curr_ :   salaryStr = float(salaryStr)*75
    elif 'KZT' in curr_:  salaryStr = float(salaryStr)*0.17
    elif 'грн' in curr_:  salaryStr = float(salaryStr)*2.6
    num =salaryStr 
    return salaryStr


# Данные о городе
def getParameterCity(arg):
  millionCities = "Новосибирск Екатеринбург Нижний Новгород Казань Челябинск Омск Самара Ростов-на-Дону Уфа Красноярск Пермь Воронеж Волгоград Сочи"
  sarg=arg.split(',')
  item = sarg[0] # Берем только город
  item= item.strip() # Удаляем пробелы
  if item == "Москва":          return "Москва"
  if item == "Санкт-Петербург": return "Санкт-Петербург"
  if item in millionCities:     return item
  else: return "малый или за пределами России город"

# Данные о желаемой занятости
def getParameterEmployment(arg):
  result = ''
  if ("стажировка" in arg)|("volunteer" in arg)|("work placement" in arg): result += 'стажировка '
  if ("частичная занятость" in arg)|("part" in arg):  result += 'частичная занятость '
  if ("проектная работа" in arg)|("project" in arg):  result += 'проектная работа '
  if ("полная занятость" in arg)|("full" in arg):     result += 'полная занятость'
  if result=='':
    result='Не указано'
  return result

# Данные о желаемом графике работы
def getParameterSchedule(arg):
  result = ''
  if ("гибкий график" in arg)|("flexible" in arg):    result += 'гибкий график '
  if ("полный день" in arg)|("full" in arg):     result += 'полный день '
  if ("сменный график" in arg)|("shift" in arg):   result += 'сменный график '
  if ("удаленная работа" in arg)|("remote" in arg): result += 'удаленная работа'
  if result=='':
    result='Не указано'
  return result

# Данные об образовании
def getParameterEducation(arg):
  result = ''
  arg = arg.lower()
  if ("высшее" in arg)|("higher education"  in arg): result += 'высшее '
  if ("среднее специальное"  in arg) | ("special"  in arg) | ("колледж"  in arg) | ("college"  in arg) : result += 'среднее специальное '
  if ("неоконченное высшее"  in arg) | ("incomplete"  in arg): result += 'неоконченное высшее '
  if ("среднее образование"  in arg) | ("secondary education"  in arg): result += 'среднее образование'
  if result=='':
    result='Не указано'
  return result

# Данные об опыте работы
def getParameterExperience(arg):
  sarg=arg.split(" ", 5)
  years = 0
  months = 0
  if len(sarg)!=6: sarg.append(0);sarg.append('');sarg.append(0);sarg.append('')
  try:
    #print(sarg)
    years = int(sarg[2])
  except ValueError:
    years = 0

  try:
    months = int(sarg[4])
  except ValueError:
    months = 0
  return 12*years+months

In [ ]:
fixed_df.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 42года , родился 6 октября 1976",27000 руб.,Старший администратор,"Липецк , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 6 лет 1 месяц Старший администрат...,Трц Ривьера,Старший администратор по сохраности активов,Среднее специальное образование 1994 Спту 26,28.04.2019 12:17,Имеется собственный автомобиль
1,"Мужчина , 41год , родился 14 января 1978",60000 руб.,Старший IT инженер,"Сочи , не готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день",Опыт работы 19 лет 2 месяца Старший IT инжене...,"Эскорт-Центр, Москва, ЗАО",Старший инженер сервисного центра,Высшее образование 1999 СОЧИНСКИЙ ГОС. УНИВЕР...,28.04.2019 11:44,Не указано
2,"Мужчина , 44года , родился 21 февраля 1975",65000 руб.,"Системный администратор, Системный инженер","Липецк , готов к переезду , готов к командировкам",полная занятость,"удаленная работа, полный день, вахтовый метод",Опыт работы 19 лет 9 месяцев Системный админи...,"СПССПК ""ЭКОПТИЦА""",Системный администратор,Высшее образование 2003 Московский государств...,28.04.2019 10:36,Не указано
3,"Мужчина , 43года , родился 9 февраля 1976",70000 руб.,"Web-программист, Web-администратор, Web-мастер","Москва , м. Марьино , не готов к переезду , г...",полная занятость,"удаленная работа, гибкий график, полный день","Опыт работы 5 лет 9 месяцев Web-программист, ...","ГБУ ""МДОО""",специалист отдела интернет-проектов,Высшее образование 2002 Российский государств...,28.04.2019 10:29,Не указано
4,"Мужчина , 39лет , родился 14 июня 1979",45000 руб.,Старший системный администратор \ начальник от...,"Тула , не готов к переезду , готов к редким ко...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 5 месяцев Старший системны...,"ООО ""СервисСофт""",Старший системный администратор,Высшее образование 2007 Российский государств...,28.04.2019 10:15,Не указано


In [ ]:
df = fixed_df.drop(fixed_df.columns[[2,7,8,10,11]],axis=1 ) # Удаляем неинтересующие нас столбцы

In [ ]:
df.head()

,"Пол, возраст",ЗП,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ
0,"Мужчина , 42года , родился 6 октября 1976",27000 руб.,"Липецк , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 6 лет 1 месяц Старший администрат...,Среднее специальное образование 1994 Спту 26
1,"Мужчина , 41год , родился 14 января 1978",60000 руб.,"Сочи , не готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день",Опыт работы 19 лет 2 месяца Старший IT инжене...,Высшее образование 1999 СОЧИНСКИЙ ГОС. УНИВЕР...
2,"Мужчина , 44года , родился 21 февраля 1975",65000 руб.,"Липецк , готов к переезду , готов к командировкам",полная занятость,"удаленная работа, полный день, вахтовый метод",Опыт работы 19 лет 9 месяцев Системный админи...,Высшее образование 2003 Московский государств...
3,"Мужчина , 43года , родился 9 февраля 1976",70000 руб.,"Москва , м. Марьино , не готов к переезду , г...",полная занятость,"удаленная работа, гибкий график, полный день","Опыт работы 5 лет 9 месяцев Web-программист, ...",Высшее образование 2002 Российский государств...
4,"Мужчина , 39лет , родился 14 июня 1979",45000 руб.,"Тула , не готов к переезду , готов к редким ко...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 5 месяцев Старший системны...,Высшее образование 2007 Российский государств...


In [ ]:
# Создаем список в которых лежат объекты наших функций
funcList = [getParameterSexAge,getParameterSalary, getParameterCity,getParameterEmployment,getParameterSchedule,getParameterExperience,getParameterEducation]

# Проходимся по каждой колонке и применяем к каждой из колонок соответствующую функцию
for i in range(len(df.columns)):
  df[df.columns[i]] =[ funcList[i](cell) for cell in df[df.columns[i]]]


In [ ]:
df.head()

,"Пол, возраст",ЗП,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ
0,"(М, 44)",27000.0,малый или за пределами России город,полная занятость,полный день,73,среднее специальное
1,"(М, 42)",60000.0,Сочи,частичная занятость полная занятость,гибкий график полный день,230,высшее
2,"(М, 45)",65000.0,малый или за пределами России город,полная занятость,полный день удаленная работа,237,высшее
3,"(М, 44)",70000.0,Москва,полная занятость,гибкий график полный день удаленная работа,69,высшее
4,"(М, 41)",45000.0,малый или за пределами России город,частичная занятость полная занятость,полный день сменный график,161,высшее


In [ ]:
df.insert(0,'Пол', np.array([df['Пол, возраст'][i][0] for i in range(len(df))]))          # Создаем отдельную колонку под Пол 
df.insert(1,'Возраст', np.array([int(df['Пол, возраст'][i][1]) for i in range(len(df))])) # Создаем отдельную колонку под Возраст
df.drop('Пол, возраст', axis = 1, inplace = True)                                         # Убираем лишнюю колонку, сочетающую в себе эти два признака

In [ ]:
df.head()

,Пол,Возраст,ЗП,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ
0,М,44,27000.0,малый или за пределами России город,полная занятость,полный день,73,среднее специальное
1,М,42,60000.0,Сочи,частичная занятость полная занятость,гибкий график полный день,230,высшее
2,М,45,65000.0,малый или за пределами России город,полная занятость,полный день удаленная работа,237,высшее
3,М,44,70000.0,Москва,полная занятость,гибкий график полный день удаленная работа,69,высшее
4,М,41,45000.0,малый или за пределами России город,частичная занятость полная занятость,полный день сменный график,161,высшее


In [ ]:
df.to_csv('/content/drive/My Drive/Базы/hh_parsed.csv', index=False)  # Сохраняем распаршенную базу

In [ ]:
fixed_df['Город'][2444]

'Нижний Новгород ,  м. Горьковская , не готова к переезду , готова к редким командировкам'

In [ ]:
f = 'Мама мыла раму, всем привет, как Ваши дела?'
f = f.split(',')
f[2]

' как Ваши дела?'

In [ ]:
sarg=fixed_df['Город'][1].split(',')
fixed_df['Город'][1]


'Сочи , не готов к переезду , готов к командировкам'

In [ ]:
sarg[1]

' не готов к переезду '

In [ ]:
df['Переезд'] = 'Нет данных'
df

,Пол,Возраст,ЗП,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ,Переезд
0,М,44,27000.0,малый или за пределами России город,полная занятость,полный день,73,среднее специальное,Нет данных
1,М,42,60000.0,Сочи,частичная занятость полная занятость,гибкий график полный день,230,высшее,Нет данных
2,М,45,65000.0,малый или за пределами России город,полная занятость,полный день удаленная работа,237,высшее,Нет данных
3,М,44,70000.0,Москва,полная занятость,гибкий график полный день удаленная работа,69,высшее,Нет данных
4,М,41,45000.0,малый или за пределами России город,частичная занятость полная занятость,полный день сменный график,161,высшее,Нет данных
...,...,...,...,...,...,...,...,...,...
66940,М,24,50000.0,малый или за пределами России город,частичная занятость проектная работа полная за...,гибкий график полный день сменный график удале...,12,среднее образование,Нет данных
66941,М,28,120000.0,Москва,полная занятость,полный день,93,высшее,Нет данных
66942,М,56,80000.0,Санкт-Петербург,частичная занятость проектная работа полная за...,гибкий график полный день удаленная работа,414,высшее,Нет данных
66943,Ж,33,70000.0,Самара,полная занятость,полный день,110,высшее,Нет данных


In [ ]:
fixed_df

,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 42года , родился 6 октября 1976",27000 руб.,Старший администратор,"Липецк , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 6 лет 1 месяц Старший администрат...,Трц Ривьера,Старший администратор по сохраности активов,Среднее специальное образование 1994 Спту 26,28.04.2019 12:17,Имеется собственный автомобиль
1,"Мужчина , 41год , родился 14 января 1978",60000 руб.,Старший IT инженер,"Сочи , не готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день",Опыт работы 19 лет 2 месяца Старший IT инжене...,"Эскорт-Центр, Москва, ЗАО",Старший инженер сервисного центра,Высшее образование 1999 СОЧИНСКИЙ ГОС. УНИВЕР...,28.04.2019 11:44,Не указано
2,"Мужчина , 44года , родился 21 февраля 1975",65000 руб.,"Системный администратор, Системный инженер","Липецк , готов к переезду , готов к командировкам",полная занятость,"удаленная работа, полный день, вахтовый метод",Опыт работы 19 лет 9 месяцев Системный админи...,"СПССПК ""ЭКОПТИЦА""",Системный администратор,Высшее образование 2003 Московский государств...,28.04.2019 10:36,Не указано
3,"Мужчина , 43года , родился 9 февраля 1976",70000 руб.,"Web-программист, Web-администратор, Web-мастер","Москва , м. Марьино , не готов к переезду , г...",полная занятость,"удаленная работа, гибкий график, полный день","Опыт работы 5 лет 9 месяцев Web-программист, ...","ГБУ ""МДОО""",специалист отдела интернет-проектов,Высшее образование 2002 Российский государств...,28.04.2019 10:29,Не указано
4,"Мужчина , 39лет , родился 14 июня 1979",45000 руб.,Старший системный администратор \ начальник от...,"Тула , не готов к переезду , готов к редким ко...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 5 месяцев Старший системны...,"ООО ""СервисСофт""",Старший системный администратор,Высшее образование 2007 Российский государств...,28.04.2019 10:15,Не указано
...,...,...,...,...,...,...,...,...,...,...,...,...
66940,"Мужчина , 22года , родился 4 сентября 1996",50000 руб.,Программист Python,"Иркутск , хочу переехать (Москва, Россия, Друг...","частичная занятость, проектная работа, полная...","гибкий график, полный день, сменный график, у...",Опыт работы 1 год Июнь2018 — Август 2018 3 ме...,Индивидуальное предпринимательство / частная п...,Программист Python,Среднее образование 2012 МБОУ СОШ №7,05.04.2019 16:17,Не указано
66941,"Мужчина , 26лет , родился 2 декабря 1992",120000 руб.,Руководитель отдела по работе с ключевыми клие...,"Москва , м. Новогиреево , готов к переезду (С...",полная занятость,полный день,Опыт работы 7 лет 9 месяцев Апрель2018 — пона...,RDP Group,Региональный менеджер по работе с ключевыми кл...,Высшее образование 2016 Национальный институт...,05.04.2019 16:17,Не указано
66942,"Мужчина , 55лет , родился 21 февраля 1964",80000 руб.,программист,"Санкт-Петербург , м. Гражданский проспект , г...","частичная занятость, проектная работа, полная...","гибкий график, полный день, удаленная работа",Опыт работы 34 года 6 месяцев Декабрь1984 — п...,ВНИИЭФ,Ведущий инженер-программист,Высшее образование 1989 Московский Инженерно-ф...,05.04.2019 16:14,Имеется собственный автомобиль
66943,"Женщина , 32года , родилась 23 февраля 1987",70000 руб.,Руководитель проектов,"Самара , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 9 лет 2 месяца Май2014 — понастоя...,МегаФон,Старший менеджер по развитию систем самообслуж...,Высшее образование 2012 Самарская гуманитарна...,05.04.2019 16:13,Не указано


## Сcылка на регулярные выражения https://tproger.ru/translations/regular-expression-python/

**Добавляем колонку "Переезд"**

In [ ]:
# Добавляем колонку "Переезд"

for i in range(len(fixed_df['Город'])):
  sarg=fixed_df['Город'][i].split(',')
  for id, j in enumerate(sarg):
    if re.search(r'перее', j):
      df['Переезд'][i] = sarg[id]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
df

,Пол,Возраст,ЗП,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ,Переезд
0,М,44,27000.0,малый или за пределами России город,полная занятость,полный день,73,среднее специальное,не готов к переезду
1,М,42,60000.0,Сочи,частичная занятость полная занятость,гибкий график полный день,230,высшее,не готов к переезду
2,М,45,65000.0,малый или за пределами России город,полная занятость,полный день удаленная работа,237,высшее,готов к переезду
3,М,44,70000.0,Москва,полная занятость,гибкий график полный день удаленная работа,69,высшее,не готов к переезду
4,М,41,45000.0,малый или за пределами России город,частичная занятость полная занятость,полный день сменный график,161,высшее,не готов к переезду
...,...,...,...,...,...,...,...,...,...
66940,М,24,50000.0,малый или за пределами России город,частичная занятость проектная работа полная за...,гибкий график полный день сменный график удале...,12,среднее образование,хочу переехать (Москва
66941,М,28,120000.0,Москва,полная занятость,полный день,93,высшее,готов к переезду (Санкт-Петербург)
66942,М,56,80000.0,Санкт-Петербург,частичная занятость проектная работа полная за...,гибкий график полный день удаленная работа,414,высшее,готов к переезду (Санкт-Петербург)
66943,Ж,33,70000.0,Самара,полная занятость,полный день,110,высшее,не готова к переезду


**Добавляем колонку "Командировки"**

In [ ]:
# Добавляем колонку "Командировки"
df['Командировки'] = 'Нет данных'
for i in range(len(fixed_df['Город'])):
  sarg=fixed_df['Город'][i].split(',')
  for id, j in enumerate(sarg):
    if re.search(r'команд', j):
      df['Командировки'][i] = sarg[id]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
df

,Пол,Возраст,ЗП,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ,Переезд,Командировки
0,М,44,27000.0,малый или за пределами России город,полная занятость,полный день,73,среднее специальное,не готов к переезду,не готов к командировкам
1,М,42,60000.0,Сочи,частичная занятость полная занятость,гибкий график полный день,230,высшее,не готов к переезду,готов к командировкам
2,М,45,65000.0,малый или за пределами России город,полная занятость,полный день удаленная работа,237,высшее,готов к переезду,готов к командировкам
3,М,44,70000.0,Москва,полная занятость,гибкий график полный день удаленная работа,69,высшее,не готов к переезду,готов к редким командировкам
4,М,41,45000.0,малый или за пределами России город,частичная занятость полная занятость,полный день сменный график,161,высшее,не готов к переезду,готов к редким командировкам
...,...,...,...,...,...,...,...,...,...,...
66940,М,24,50000.0,малый или за пределами России город,частичная занятость проектная работа полная за...,гибкий график полный день сменный график удале...,12,среднее образование,хочу переехать (Москва,готов к командировкам
66941,М,28,120000.0,Москва,полная занятость,полный день,93,высшее,готов к переезду (Санкт-Петербург),готов к командировкам
66942,М,56,80000.0,Санкт-Петербург,частичная занятость проектная работа полная за...,гибкий график полный день удаленная работа,414,высшее,готов к переезду (Санкт-Петербург),готов к командировкам
66943,Ж,33,70000.0,Самара,полная занятость,полный день,110,высшее,не готова к переезду,готова к командировкам


In [ ]:
df['Командировки'].value_counts()

 готов к командировкам            24854
 не готов к командировкам         15354
 готов к редким командировкам     12366
 не готова к командировкам         5031
 готова к командировкам            4445
 готова к редким командировкам     3005
Нет данных                         1885
 готов к команд                       3
 готов к командир                     1
 готов к командировка                 1
Name: Командировки, dtype: int64

In [ ]:
df

,Пол,Возраст,ЗП,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ,Переезд,Командировки
0,М,44,27000.0,малый или за пределами России город,полная занятость,полный день,73,среднее специальное,не готов к переезду,не готов к командировкам
1,М,42,60000.0,Сочи,частичная занятость полная занятость,гибкий график полный день,230,высшее,не готов к переезду,готов к командировкам
2,М,45,65000.0,малый или за пределами России город,полная занятость,полный день удаленная работа,237,высшее,готов к переезду,готов к командировкам
3,М,44,70000.0,Москва,полная занятость,гибкий график полный день удаленная работа,69,высшее,не готов к переезду,готов к редким командировкам
4,М,41,45000.0,малый или за пределами России город,частичная занятость полная занятость,полный день сменный график,161,высшее,не готов к переезду,готов к редким командировкам
...,...,...,...,...,...,...,...,...,...,...
66940,М,24,50000.0,малый или за пределами России город,частичная занятость проектная работа полная за...,гибкий график полный день сменный график удале...,12,среднее образование,хочу переехать (Москва,готов к командировкам
66941,М,28,120000.0,Москва,полная занятость,полный день,93,высшее,готов к переезду (Санкт-Петербург),готов к командировкам
66942,М,56,80000.0,Санкт-Петербург,частичная занятость проектная работа полная за...,гибкий график полный день удаленная работа,414,высшее,готов к переезду (Санкт-Петербург),готов к командировкам
66943,Ж,33,70000.0,Самара,полная занятость,полный день,110,высшее,не готова к переезду,готова к командировкам


# Задание. База с Юлы

**Загрузка данных**

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Базы/moscow.csv', sep=";") # Загружаем данные в data frame
df # Отображаем данные

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",ГРМ,Бонус агенту,Дата,Кол-во дней в экспозиции,Источник,Примечание
0,1,Шелепиха м.,12п,35/37 М,NaN,2,64.1/23/20,19500000.0,NaN,NaN,25.07.2019,1.0,Привлекательные,Собственность оформлена! Ключи на руках! Опера...
1,Для заметок:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,NaN,NaN,25.07.2019,2.0,Привлекательные,Зеленоград к.1820 однокомнатная квартира общ.3...
3,Для заметок:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,NaN,NaN,25.07.2019,4.0,Привлекательные,"Уютная, светлая квартира, застекленный балкон ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252531,Для заметок:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252532,Ст,Алтуфьево м.,5т,4/9 М,NaN,NaN,26/10.9/7,5200000.0,NaN,NaN,22.07.2019,14.0,AVITO.ru,Продается студия с отделкой в новом монолитном...
252533,Для заметок:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252534,Ст,Волжская м.,10п,1/5 К,NaN,NaN,17.5/15.2/?,2790000.0,NaN,NaN,25.07.2019,14.0,AVITO.ru,ДАМЫ И ГОСПОДА! ВАШЕМУ ВНИМАНИЮ ПРЕДЛАГАЕТСЯ С...


**1. Убираем строки "Для заметок"**

In [ ]:
df = df[df['Комнат'] != 'Для заметок:'] # Оставляем в таблице только те строки, у которых в графе "Комнат" нет записи "Для заметок"
df # Смотрим результат

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",ГРМ,Бонус агенту,Дата,Кол-во дней в экспозиции,Источник,Примечание
0,1,Шелепиха м.,12п,35/37 М,NaN,2,64.1/23/20,19500000.0,NaN,NaN,25.07.2019,1.0,Привлекательные,Собственность оформлена! Ключи на руках! Опера...
2,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,NaN,NaN,25.07.2019,2.0,Привлекательные,Зеленоград к.1820 однокомнатная квартира общ.3...
4,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,NaN,NaN,25.07.2019,4.0,Привлекательные,"Уютная, светлая квартира, застекленный балкон ..."
6,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200000.0,NaN,NaN,25.07.2019,7.0,Привлекательные,"Предлагаю 1- комн квартиру, в шаговой доступно..."
8,1,Бабушкинская м.,15п,4/14 П,NaN,С,34/17/8,6250000.0,NaN,NaN,25.07.2019,11.0,Привлекательные,Продается хорошая 1 комнатная квартира. 1 взро...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252526,Ст,Некрасовка м.,4п,2/17 П,NaN,NaN,19.8/?/?,2850000.0,NaN,NaN,12.07.2019,14.0,AVITO.ru,Уникальнейшее предложение! 7 минуты до метро! ...
252528,Ст,Авиамоторная м.,6п,2/8 К,NaN,С,20.4/18/?,3839990.0,NaN,NaN,23.07.2019,14.0,Яндекс,ВЫГОДНОЕ ПРЕДЛОЖЕНИЕ в пешей доступности от ме...
252530,Ст,Коломенская м.,15п,1/9 П,NaN,NaN,15.4/13.2/?,2900000.0,NaN,NaN,12.07.2019,14.0,AVITO.ru,СТУДИЯ В МОСКВЕ ПО ДОСТУПНОЙ ЦЕНЕ ! __________...
252532,Ст,Алтуфьево м.,5т,4/9 М,NaN,NaN,26/10.9/7,5200000.0,NaN,NaN,22.07.2019,14.0,AVITO.ru,Продается студия с отделкой в новом монолитном...


In [ ]:
df = df.reset_index(drop = True) # Переназначим индекс с удалением предыдущего индекса
df # Смотрим результат

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",ГРМ,Бонус агенту,Дата,Кол-во дней в экспозиции,Источник,Примечание
0,1,Шелепиха м.,12п,35/37 М,NaN,2,64.1/23/20,19500000.0,NaN,NaN,25.07.2019,1.0,Привлекательные,Собственность оформлена! Ключи на руках! Опера...
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,NaN,NaN,25.07.2019,2.0,Привлекательные,Зеленоград к.1820 однокомнатная квартира общ.3...
2,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,NaN,NaN,25.07.2019,4.0,Привлекательные,"Уютная, светлая квартира, застекленный балкон ..."
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200000.0,NaN,NaN,25.07.2019,7.0,Привлекательные,"Предлагаю 1- комн квартиру, в шаговой доступно..."
4,1,Бабушкинская м.,15п,4/14 П,NaN,С,34/17/8,6250000.0,NaN,NaN,25.07.2019,11.0,Привлекательные,Продается хорошая 1 комнатная квартира. 1 взро...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126263,Ст,Некрасовка м.,4п,2/17 П,NaN,NaN,19.8/?/?,2850000.0,NaN,NaN,12.07.2019,14.0,AVITO.ru,Уникальнейшее предложение! 7 минуты до метро! ...
126264,Ст,Авиамоторная м.,6п,2/8 К,NaN,С,20.4/18/?,3839990.0,NaN,NaN,23.07.2019,14.0,Яндекс,ВЫГОДНОЕ ПРЕДЛОЖЕНИЕ в пешей доступности от ме...
126265,Ст,Коломенская м.,15п,1/9 П,NaN,NaN,15.4/13.2/?,2900000.0,NaN,NaN,12.07.2019,14.0,AVITO.ru,СТУДИЯ В МОСКВЕ ПО ДОСТУПНОЙ ЦЕНЕ ! __________...
126266,Ст,Алтуфьево м.,5т,4/9 М,NaN,NaN,26/10.9/7,5200000.0,NaN,NaN,22.07.2019,14.0,AVITO.ru,Продается студия с отделкой в новом монолитном...


**2. Удаляем столбцы "Примечание" и "ГРМ"**

In [ ]:
del df['Примечание'] # Удаляем "Примечание"

In [ ]:
del df['ГРМ'] # Удаляем "ГРМ"

In [ ]:
df # Смотрим результат

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,NaN,2,64.1/23/20,19500000.0,NaN,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,NaN,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,NaN,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200000.0,NaN,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,NaN,С,34/17/8,6250000.0,NaN,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
126263,Ст,Некрасовка м.,4п,2/17 П,NaN,NaN,19.8/?/?,2850000.0,NaN,12.07.2019,14.0,AVITO.ru
126264,Ст,Авиамоторная м.,6п,2/8 К,NaN,С,20.4/18/?,3839990.0,NaN,23.07.2019,14.0,Яндекс
126265,Ст,Коломенская м.,15п,1/9 П,NaN,NaN,15.4/13.2/?,2900000.0,NaN,12.07.2019,14.0,AVITO.ru
126266,Ст,Алтуфьево м.,5т,4/9 М,NaN,NaN,26/10.9/7,5200000.0,NaN,22.07.2019,14.0,AVITO.ru


**3. Для столбца "Балкон" вместо Nan-значений создаем категорию "Отсутсвует"**

In [ ]:
df.loc[df['Балкон'].isna(), 'Балкон'] = 'Отсутствует'
df # Смотрим результат

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1/23/20,19500000.0,NaN,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,NaN,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,NaN,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200000.0,NaN,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34/17/8,6250000.0,NaN,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
126263,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,NaN,19.8/?/?,2850000.0,NaN,12.07.2019,14.0,AVITO.ru
126264,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20.4/18/?,3839990.0,NaN,23.07.2019,14.0,Яндекс
126265,Ст,Коломенская м.,15п,1/9 П,Отсутствует,NaN,15.4/13.2/?,2900000.0,NaN,12.07.2019,14.0,AVITO.ru
126266,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,NaN,26/10.9/7,5200000.0,NaN,22.07.2019,14.0,AVITO.ru


**4. Значения столбца "Бонус агенту" переводим в тысячи рублей, остальные значения заполняем нулями**

In [ ]:
df.loc[df['Бонус агенту'].isna(), 'Бонус агенту'] = 0 # Зададим значение 0 для Nan-значений

In [ ]:
bonus = df['Бонус агенту'] # Возьмем столбец "Бонус агенту"
cost = df['Цена, руб.']    # Возьмем столбец 'Цена, руб.'

In [ ]:
pd.options.mode.chained_assignment = None # Отключаем warnings

for i in range(bonus.shape[0]): # Пройдем по всем значениям столбца "Бонус агенту"

  # Если значение является строкой (не 0, где раньше был Nan) и в нем есть 
  # подстрока 'руб.', значит в столбце явно указана сумма ганорара
  if isinstance(bonus[i], str) and bonus[i].find('руб.') != -1:  
    bonus[i] = bonus[i][:bonus[i].find('руб.')] # Оставляем только численное значение ганорара 
    bonus[i] = bonus[i].replace(' ', '')        # Удаляем пробелы,
    bonus[i] = float(bonus[i])                  # Переводим в float
    bonus[i] = bonus[i] / 1000                  # Делим на 1000

  # Если значение является строкой (не 0, где раньше был Nan) и в нем есть 
  # подстрока '%', значит в столбце ганорар указан в процентах от сделки
  elif isinstance(bonus[i], str) and bonus[i].find('%') != -1: 
    bonus[i] = bonus[i][:bonus[i].find('%')] # Оставляем только процентное значение ганорара 
    bonus[i] = bonus[i].replace(' ', '')     # Удаляем пробелы,
    bonus[i] = float(bonus[i])               # Переводим в float
    bonus[i] = bonus[i] / 100                # Делим на 100 (так как это проценты)
    bonus[i] = bonus[i] * cost[i]            # Умножаем на значение сделки
    bonus[i] = bonus[i] / 1000               # Делим на 1000
    
df['Бонус агенту'] = bonus # Сформированный массив присваиваем к столбцу
df # Смотрим результат

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1/23/20,19500000.0,0,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,0,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,0,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200000.0,0,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34/17/8,6250000.0,0,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
126263,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,NaN,19.8/?/?,2850000.0,0,12.07.2019,14.0,AVITO.ru
126264,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20.4/18/?,3839990.0,0,23.07.2019,14.0,Яндекс
126265,Ст,Коломенская м.,15п,1/9 П,Отсутствует,NaN,15.4/13.2/?,2900000.0,0,12.07.2019,14.0,AVITO.ru
126266,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,NaN,26/10.9/7,5200000.0,0,22.07.2019,14.0,AVITO.ru


In [ ]:
df.rename(columns={'Бонус агенту': 'Бонус агенту, тыс. руб.'}, inplace=True) # Даем новое название столбцу
df # Смотрим результат

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.","Бонус агенту, тыс. руб.",Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1/23/20,19500000.0,0,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,0,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,0,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200000.0,0,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34/17/8,6250000.0,0,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
126263,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,NaN,19.8/?/?,2850000.0,0,12.07.2019,14.0,AVITO.ru
126264,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20.4/18/?,3839990.0,0,23.07.2019,14.0,Яндекс
126265,Ст,Коломенская м.,15п,1/9 П,Отсутствует,NaN,15.4/13.2/?,2900000.0,0,12.07.2019,14.0,AVITO.ru
126266,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,NaN,26/10.9/7,5200000.0,0,22.07.2019,14.0,AVITO.ru


In [ ]:
df.loc[df['Бонус агенту, тыс. руб.'] != 0, 'Бонус агенту, тыс. руб.'] # Посмотрим на ненулевые значения

152          10
304          10
319          10
388          50
456        70.5
          ...  
124244       20
124510        5
124753    10.15
125287      150
125843       50
Name: Бонус агенту, тыс. руб., Length: 1252, dtype: object

**5. Считаем в цикле суммарную площадь помещений**

In [ ]:
# Функция получения суммарной площади числа. На вход передаем строку столбца "Площадь". 
# В этом столбце попадаются записи, не соответсвующие формату, например, в некоторых ячейках имеется дата, хотя это столбец "Площадь"
# Суммарное значение площади может быть неизвестно. В этом случае будем вносить в таблицу 0

def parcer(strg):
  result = 0                            # Создаем возвращаемую переменную и приравниваем к 0
  if strg.find('/') != -1:              # Если в строке есть '/'
    strg = strg[:strg.find('/')]        # Берем все, что находится до него
    search=re.compile(r'[0-9.]').search # Создаем элемент для поиска недопустимых символов 
    if bool(search(strg)): # Проверяем наличие недопустимых символов. Если есть, то coorectSymbols = False, иначе True 
      result = float(strg) # Преобразуем в float

  return result            # Возвращаем результат 

areas = df['Площадь']      # Берем все записи в столбце "Площадь" 
finalAreas = []            # Создаем новый список, в который будем добавлять площади
for i in range(areas.shape[0]):         # Проходим по всем строчкам столбца
  finalAreas.append(parcer(areas[i]))   # Добавляем в список значение площади

df['Площадь'] = finalAreas # Меняем значения площадей помещений
df # Смотрим результат

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.","Бонус агенту, тыс. руб.",Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1,19500000.0,0,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4,4800000.0,0,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,38.2,7990000.0,0,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38.0,11200000.0,0,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34.0,6250000.0,0,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
126263,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,NaN,19.8,2850000.0,0,12.07.2019,14.0,AVITO.ru
126264,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20.4,3839990.0,0,23.07.2019,14.0,Яндекс
126265,Ст,Коломенская м.,15п,1/9 П,Отсутствует,NaN,15.4,2900000.0,0,12.07.2019,14.0,AVITO.ru
126266,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,NaN,26.0,5200000.0,0,22.07.2019,14.0,AVITO.ru


**6. Отсутствующие значения в столбце "Санузел" меняем на "Нет данных"**

In [ ]:
df.loc[df['Санузел'].isna(), 'Санузел'] = 'Нет данных'
df # Смотрим результат

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.","Бонус агенту, тыс. руб.",Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1,19500000.0,0,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4,4800000.0,0,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,38.2,7990000.0,0,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38.0,11200000.0,0,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34.0,6250000.0,0,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
126263,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,Нет данных,19.8,2850000.0,0,12.07.2019,14.0,AVITO.ru
126264,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20.4,3839990.0,0,23.07.2019,14.0,Яндекс
126265,Ст,Коломенская м.,15п,1/9 П,Отсутствует,Нет данных,15.4,2900000.0,0,12.07.2019,14.0,AVITO.ru
126266,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,Нет данных,26.0,5200000.0,0,22.07.2019,14.0,AVITO.ru


**7. Переводим значение столбца 'Цена, руб.' В тысячи**

In [ ]:
df['Цена, руб.'] = df['Цена, руб.'] / 1000

In [ ]:
df.rename(columns={'Цена, руб.': 'Цена, тыс. руб.'}, inplace=True) # Даем новое название столбцу
df # Смотрим результат

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, тыс. руб.","Бонус агенту, тыс. руб.",Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1,19500.00,0,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4,4800.00,0,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,38.2,7990.00,0,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38.0,11200.00,0,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34.0,6250.00,0,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
126263,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,Нет данных,19.8,2850.00,0,12.07.2019,14.0,AVITO.ru
126264,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20.4,3839.99,0,23.07.2019,14.0,Яндекс
126265,Ст,Коломенская м.,15п,1/9 П,Отсутствует,Нет данных,15.4,2900.00,0,12.07.2019,14.0,AVITO.ru
126266,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,Нет данных,26.0,5200.00,0,22.07.2019,14.0,AVITO.ru


In [ ]:
df['Дата'].unique()

array(['25.07.2019', '24.07.2019', '23.07.2019', '22.07.2019',
       '21.07.2019', '20.07.2019', '18.07.2019', '17.07.2019',
       '16.07.2019', '15.07.2019', '14.07.2019', '12.07.2019',
       '19.07.2019', '13.07.2019', '11.07.2019', '10.07.2019',
       '08.07.2019', '05.07.2019', '03.07.2019', '01.07.2019',
       '07.07.2019', '09.07.2019', '25.06.2019', '21.06.2019',
       '20.06.2019', '04.07.2019', '30.06.2019', '28.06.2019',
       '14.06.2019', '06.06.2019', '26.06.2019', '05.06.2019',
       '02.07.2019', '18.06.2019', '23.06.2019', '24.06.2019',
       '27.06.2019', '09.06.2019', '06.07.2019', '19.06.2019',
       '13.06.2019', '17.06.2019', '29.06.2019', '22.06.2019',
       '12.06.2019', '10.06.2019', '08.06.2019', '07.06.2019',
       '11.06.2019', '16.06.2019', '15.06.2019', '26.07.2019',
       '03.08.2019', '02.08.2019', '01.08.2019', '31.07.2019',
       '30.07.2019', '29.07.2019', '28.07.2019', '27.07.2019'],
      dtype=object)